In [ ]:
# Execute when using Colab for loading the dataset from my Github account and creating the different subsets

'''
! git clone https://github.com/Franborat/ml-datasets.git
  
! cd ml-datasets/

import pandas as pd

ATT_FILE = "MedianHouseValuePreparedCleanAttributes.csv"
LABEL_FILE = "MedianHouseValueOneHotEncodedClasses.csv"

TRAIN_RATE = 0.8

attributes = pd.read_csv(ATT_FILE)
label = pd.read_csv(LABEL_FILE)

n_instances = attributes.shape[0]
n_train = int(n_instances*TRAIN_RATE)
n_dev = int((n_instances-n_train)/2)
n_test = n_instances-n_train-n_dev

x_train = attributes.values[:n_train]
t_train = label.values[:n_train]

x_dev = attributes.values[n_train:n_train+n_dev]
t_dev = label.values[n_train:n_train+n_dev]

x_test = attributes.values[n_train+n_dev:n_instances]
t_test = label.values[n_train+n_dev:n_instances]

print ("x_train:",x_train.shape)
print ("t_train:",t_train.shape)

print ("x_dev:",x_dev.shape)
print ("t_dev:",t_dev.shape)

print ("x_test:",x_test.shape)
print ("t_test:",t_test.shape)


In [ ]:
# Load data
%run 1.ReadingData.py

In [ ]:
INPUTS = x_train.shape[1]
OUTPUTS = t_train.shape[1]
NUM_TRAINING_EXAMPLES = int(round(x_train.shape[0]/1))
NUM_DEV_EXAMPLES = int (round (x_dev.shape[0]/1))
NUM_TEST_EXAMPLES = int (round (x_test.shape[0]/1))

## HyperParameters

In [ ]:
from itertools import product
dense_layers = [1,2,3]
learning_rates = [0.001]
#learning_rates = [1e-06, 1e-05, 0.0001, 0.001, 0.01, 0.1]
batch_sizes = [32]
#dropout_rates = [0.1,0.2,0.3,0.4,0.5]
dropout_rates = [0.2]
neurons = [64]

# EarlyStopping
from keras.callbacks import EarlyStopping
earlystop = EarlyStopping(monitor='val_acc', patience=10)

In [ ]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
from keras import optimizers
from time import time
from keras.callbacks import TensorBoard
from keras.regularizers import l2

## GridSearch with TensorBoard Logs

In [ ]:
for dense_layer in dense_layers:
    # All combinations available
    layer_neurons = list(product(neurons, repeat=dense_layer))
    for layer_neuron in layer_neurons:
        for learning_rate in learning_rates:
            for batch_size in batch_sizes:
                for dropout_rate in dropout_rates:
                    
                    mlp = Sequential()
                    
                    mlp.add(Dense(layer_neuron[0], input_dim=INPUTS))
                    mlp.add(Activation('relu'))
                    mlp.add(Dropout(0.2))
                    
                    if(dense_layer==1):
                        NAME = "{}x-Arquitecture-{}".format(layer_neuron[0], int(time()))
                    if(dense_layer==2):
                        NAME = "{}x{}x-Arquitecture-{}".format(layer_neuron[0], layer_neuron[1], int(time()))
                    if(dense_layer==3):
                        NAME = "{}x{}x{}-Arquitecture-{}".format(layer_neuron[0], layer_neuron[1], layer_neuron[2], int(time()))
                    print(NAME)

                    for layer in range(dense_layer-1):
                        print("------{}".format(layer))
                        mlp.add(Dense(layer_neuron[layer+1]))
                        mlp.add(Activation('relu'))
                        mlp.add(Dropout(0.2))  
                        
                    # Final layer
                    mlp.add(Dense(OUTPUTS))
                    mlp.add(Dropout(0.2))
                    mlp.add(Activation('softmax'))
                    
                    tensorboard = TensorBoard(log_dir="Arquitecture-tuning/{}".format(NAME))
                    
                    
                    opt = optimizers.Adam(lr=learning_rate)
                    mlp.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])
                    mlp.summary()    
                    mlp.fit(x_train, t_train, batch_size=batch_size, epochs=50, verbose=2, validation_data=(x_dev, t_dev), callbacks=[tensorboard, earlystop])    

In [ ]:
# Execute when using Colab to Download Tensorboard files

'''
!zip -r Final100.zip Arquitecture-tuning-weirds

from google.colab import files
files.download("Final100.zip")

## Performance Measures

In [ ]:
start = time()
loss, acc = mlp.evaluate(x_test, t_test, verbose=0)
end = time()
print('MLP took ' + str(end - start) + ' seconds')
print('Test loss: ' + str(loss) + ' - Accuracy: ' + str(acc))